<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Вопрос-1" data-toc-modified-id="Вопрос-1-1">Вопрос 1</a></span></li><li><span><a href="#Вопрос-2" data-toc-modified-id="Вопрос-2-2">Вопрос 2</a></span></li><li><span><a href="#Вопрос-3" data-toc-modified-id="Вопрос-3-3">Вопрос 3</a></span></li><li><span><a href="#Вопрос-4" data-toc-modified-id="Вопрос-4-4">Вопрос 4</a></span></li><li><span><a href="#Вопрос-5" data-toc-modified-id="Вопрос-5-5">Вопрос 5</a></span></li></ul></div>

# Анализ сервиса прогнозирования временных рядов.

In [1]:
# импортируем необходимые библиотеки
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# прочитаем файлы, которые представляет собой sqlite базу данных
cnx_12 = sqlite3.connect('logs_2022-09-12.db')
cnx_13 = sqlite3.connect('logs_2022-09-13.db')
cnx_14 = sqlite3.connect('logs_2022-09-14.db')

In [3]:
event_name_12 = pd.read_sql_query("SELECT * FROM event_name", cnx_12)
events_12 = pd.read_sql_query("SELECT * FROM events", cnx_12)

#event_name_13 = pd.read_sql_query("SELECT * FROM event_name", cnx_13) # таблица повторяется
events_13 = pd.read_sql_query("SELECT * FROM events", cnx_13)

#event_name_14 = pd.read_sql_query("SELECT * FROM event_name", cnx_14) # таблица повторяется
events_14 = pd.read_sql_query("SELECT * FROM events", cnx_14)

Откроем таблицу event_name (таблица с именами событий, которые случаются в сервисе) - названия событий понадобятся для дальнейшего анализа

In [4]:
event_name_12

,id,event_name
0,1,Инициализируем свойства класса
1,2,Переход в функцию getParametrsAndRegressors
2,3,Выход из функции getParametrsAndRegressors с п...
3,4,Выход из функции getParametrsAndRegressors БЕЗ...
4,5,Записали в свойства класса гиперпараметры и ре...
5,6,Начинаем читать данные из BQ по полученному SQ...
6,7,Получили данные из BQ по полученному SQL запросу
7,8,saveForecastRequest: вход в функцию
8,9,saveForecastRequest: выход из функции
9,10,getDOW: вход в функцию


In [5]:
# приведём столбец 'request_time' к формату времени во всех таблицах
for table in (events_12, events_13, events_14):
    table['request_time'] = pd.to_datetime(table['request_time'], format="%Y-%m-%d %H:%M:%S.%f")
    print(table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11207 entries, 0 to 11206
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              11207 non-null  int64         
 1   forecastMarker  11207 non-null  object        
 2   request_time    11207 non-null  datetime64[ns]
 3   event_id        11207 non-null  int64         
 4   sender          11207 non-null  object        
 5   context         11207 non-null  object        
 6   session_id      11207 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 613.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22275 entries, 0 to 22274
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              22275 non-null  int64         
 1   forecastMarker  22275 non-null  object        
 2   request_time    22275 non-n

In [6]:
# объединим таблицы
events = events_12.append(events_13, ignore_index=True)
events = events.append(events_14, ignore_index=True)
print(events.info())
events.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44731 entries, 0 to 44730
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              44731 non-null  int64         
 1   forecastMarker  44731 non-null  object        
 2   request_time    44731 non-null  datetime64[ns]
 3   event_id        44731 non-null  int64         
 4   sender          44731 non-null  object        
 5   context         44731 non-null  object        
 6   session_id      44731 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 2.4+ MB
None


,id,forecastMarker,request_time,event_id,sender,context,session_id
0,1,SelfService/RUS/turnover,2022-09-12 11:30:11.106628,1,test_system,"df = <class 'NoneType'>, sql_query = <class 'N...",6f4277f6df57c4f4e90779966522c876
1,2,SelfService/RUS/turnover,2022-09-12 11:30:11.111359,29,test_system,None,6f4277f6df57c4f4e90779966522c876
2,3,SelfService/RUS/turnover,2022-09-12 11:30:11.115989,2,test_system,None,6f4277f6df57c4f4e90779966522c876
3,4,SelfService/RUS/turnover,2022-09-12 11:30:12.761754,3,test_system,Найдено 1 строк с параметрами,6f4277f6df57c4f4e90779966522c876
4,5,SelfService/RUS/turnover,2022-09-12 11:30:12.770134,5,test_system,None,6f4277f6df57c4f4e90779966522c876
5,6,SelfService/RUS/turnover,2022-09-12 11:30:12.780523,6,test_system,None,6f4277f6df57c4f4e90779966522c876
6,7,SelfService/RUS/turnover,2022-09-12 11:30:16.853353,7,test_system,"Получили из BQ набор данных размером: (1715, 2)",6f4277f6df57c4f4e90779966522c876
7,8,SelfService/RUS/turnover,2022-09-12 11:30:16.858601,8,test_system,None,6f4277f6df57c4f4e90779966522c876
8,9,SelfService/RUS/turnover,2022-09-12 11:30:16.866860,9,test_system,None,6f4277f6df57c4f4e90779966522c876
9,10,SelfService/RUS/turnover,2022-09-12 11:30:16.872044,10,test_system,None,6f4277f6df57c4f4e90779966522c876


Объединение таблиц произошло успешно, количество строк полученной таблицы соответсвует сумме строк первоначальных таблиц.

## Вопрос 1

**Между какими событиями наибольший шаг? Укажите пару идентификаторов
событий с наибольшим шагом, относительно всей базы (всех файлов).**

In [7]:
# создаём список идентификаторов задач
sessions_list = list(pd.unique(events['session_id']))

In [8]:
# определим переменные для цикла
event_id_1 = 0
event_id_2 = 0
t = datetime.strptime('00:00:00.000000', '%H:%M:%S.%f')
event_time_max = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
session_event_time_max = 'none'

for session in sessions_list:
    # поскольку задачи могут выполняться параллельно, выделим информацию по каждой задаче отдельно
    session_data = events.query('session_id == @session').reset_index(drop=True)
    
    # посчитаем время между каждым шагом выполнения задачи
    for index in range(0, len(session_data)-1):
        event_time = session_data.loc[index+1, 'request_time'] - session_data.loc[index, 'request_time']
        
        # сохраним информацию о максимальном шаге событий
        if event_time > event_time_max:
            event_id_1 = session_data.loc[index, 'event_id']
            event_id_2 = session_data.loc[index+1, 'event_id']
            event_time_max = event_time
            session_event_time_max = session
            
print('Наибольший шаг между событиями:', event_id_1 ,'и', event_id_2)
print('Шаг:', event_time_max)
print('Идентификатор соответствующей задачи:', session_event_time_max)

Наибольший шаг между событиями: 6 и 7
Шаг: 0 days 00:01:46.548210
Идентификатор соотвествующей задачи: 4dcab2af4573ced8251fb7433174ba15


In [9]:
# выведем всю информацию о выполнении задачи, в которой присутсвует максимальный шаг
events.query('session_id == @session_event_time_max').reset_index(drop=True)

,id,forecastMarker,request_time,event_id,sender,context,session_id
0,5738,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.203702,1,weekly_report.care,"df = <class 'NoneType'>, sql_query = <class 'N...",4dcab2af4573ced8251fb7433174ba15
1,5739,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.210420,29,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
2,5740,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.214649,2,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
3,5741,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:53.747368,3,weekly_report.care,Найдено 1 строк с параметрами,4dcab2af4573ced8251fb7433174ba15
4,5742,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:53.754025,5,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
5,5743,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:53.759918,6,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
6,5744,weekly_report.care:count_of_jivo_SA,2022-09-14 05:22:40.308128,7,weekly_report.care,"Получили из BQ набор данных размером: (548, 2)",4dcab2af4573ced8251fb7433174ba15
7,5745,weekly_report.care:count_of_jivo_SA,2022-09-14 05:22:40.312967,8,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
8,5746,weekly_report.care:count_of_jivo_SA,2022-09-14 05:22:40.319745,9,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15
9,5747,weekly_report.care:count_of_jivo_SA,2022-09-14 05:22:40.324240,10,weekly_report.care,None,4dcab2af4573ced8251fb7433174ba15


Максимальный шаг достигается в данной задаче между событиями 6 и 7: событие 6 соотвествует чтению данных из BQ по полученному SQL запросу. Данный факт, вероятно, связан со скоростью обработки запроса (продолжительное время обработки данных сервером, время на пересылку данных).

## Вопрос 2

**Интервал времени до наступления какого события показывает наибольший разброс. Укажите пару идентификатор и sender с наибольшим разбросом. Предложите свои идеи, с чем это может быть связано.**

Подготовим промежуточную таблицу для решения задачи.

In [10]:
# определим переменные для цикла
events_time_disp = pd.DataFrame()

for session in sessions_list:
    # выделим информацию по каждой задаче отдельно
    session_data = events.query('session_id == @session').reset_index(drop=True)
    
    # определим время начала выполнения задачи
    start_time = str(session_data.loc[session_data['event_id'] == 1,'request_time'][0])
    # создадим столбец, содержащий информацию о времени начала выполнения задачи
    session_data['start_time'] = pd.Series([start_time for x in range(len(session_data.index))])
    session_data['start_time'] = pd.to_datetime(session_data['start_time'], format="%Y-%m-%d %H:%M:%S.%f")
    # посчитаем время от начала выполнения задачи до конкретного события в ээтой задаче
    session_data['time_till_event'] = (session_data['request_time'] - session_data['start_time'])
    
    #соберём данные по каждой задаче в общую таблицу
    events_time_disp = events_time_disp.append(session_data).reset_index(drop=True)
    
print(events_time_disp.info()) 
events_time_disp.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44731 entries, 0 to 44730
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   id               44731 non-null  int64          
 1   forecastMarker   44731 non-null  object         
 2   request_time     44731 non-null  datetime64[ns] 
 3   event_id         44731 non-null  int64          
 4   sender           44731 non-null  object         
 5   context          44731 non-null  object         
 6   session_id       44731 non-null  object         
 7   start_time       44731 non-null  datetime64[ns] 
 8   time_till_event  44731 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), int64(2), object(4), timedelta64[ns](1)
memory usage: 3.1+ MB
None


,id,forecastMarker,request_time,event_id,sender,context,session_id,start_time,time_till_event
0,1,SelfService/RUS/turnover,2022-09-12 11:30:11.106628,1,test_system,"df = <class 'NoneType'>, sql_query = <class 'N...",6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:00
1,2,SelfService/RUS/turnover,2022-09-12 11:30:11.111359,29,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:00.004731
2,3,SelfService/RUS/turnover,2022-09-12 11:30:11.115989,2,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:00.009361
3,4,SelfService/RUS/turnover,2022-09-12 11:30:12.761754,3,test_system,Найдено 1 строк с параметрами,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:01.655126
4,5,SelfService/RUS/turnover,2022-09-12 11:30:12.770134,5,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:01.663506
5,6,SelfService/RUS/turnover,2022-09-12 11:30:12.780523,6,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:01.673895
6,7,SelfService/RUS/turnover,2022-09-12 11:30:16.853353,7,test_system,"Получили из BQ набор данных размером: (1715, 2)",6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:05.746725
7,8,SelfService/RUS/turnover,2022-09-12 11:30:16.858601,8,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:05.751973
8,9,SelfService/RUS/turnover,2022-09-12 11:30:16.866860,9,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:05.760232
9,10,SelfService/RUS/turnover,2022-09-12 11:30:16.872044,10,test_system,None,6f4277f6df57c4f4e90779966522c876,2022-09-12 11:30:11.106628,0 days 00:00:05.765416


Количество строк в таблице events_time_disp равно количеству строк в таблице events - данные нигде не потерялись.
Приступим к созданию сводной таблицы, содержащей минимальный и максимальный интервал до наступления определённого события.

In [11]:
# создадим сводную таблицу с минимальным и максимальным интервалом ожидания события
events_disp = events_time_disp.query('event_id != 1').pivot_table(index=['event_id', 'sender'],
                                  values='time_till_event', 
                                  aggfunc=['min','max'])
# переименуем столбцы сводной таблицы
events_disp.columns = ['min_time', 'max_time']
print(events_disp.info())
events_disp.head(5)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 946 entries, (2, 'DAG : bq_analytics_tables_refresh') to (34, 'weekly_report.ss')
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   min_time  946 non-null    timedelta64[ns]
 1   max_time  946 non-null    timedelta64[ns]
dtypes: timedelta64[ns](2)
memory usage: 17.3+ KB
None


min_time  \
event_id sender                                                                      
2        DAG : bq_analytics_tables_refresh                  0 days 00:00:00.008564   
         None                                               0 days 00:00:00.008644   
         forecast_for_PHR.load_prophet_forecast_data        0 days 00:00:00.009384   
         internal_call for Agency/turnover, session_id: ... 0 days 00:00:00.008934   
         internal_call for Agency/turnover, session_id: ... 0 days 00:00:00.011640   

                                                                          max_time  
event_id sender                                                                     
2        DAG : bq_analytics_tables_refresh                  0 days 00:00:00.265056  
         None                                               0 days 00:00:00.041304  
         forecast_for_PHR.load_prophet_forecast_data        0 days 00:00:00.378992  
         internal_call for Agency/turnover, session_id: ... 0 days 00:00:00.008934  
         internal_call for Agency/turnover, session_id: ... 0 days 00:00:00.011640

In [12]:
# добавим столбец с разницей между максимальным и минимальным временем до наступления события события
events_disp['time_dispersion'] = events_disp['max_time'] - events_disp['min_time']

# отсортируем таблицу по убыванию разброса времени, выведем топ-5 event_id и sender с наибольшим разбросом времени
events_disp = events_disp.sort_values(by='time_dispersion', ascending=False).reset_index()
events_disp.head(5)

,event_id,sender,min_time,max_time,time_dispersion
0,20,weekly_report.care,0 days 00:00:02.079249,0 days 00:01:50.492566,0 days 00:01:48.413317
1,25,weekly_report.care,0 days 00:00:01.779209,0 days 00:01:49.876690,0 days 00:01:48.097481
2,26,weekly_report.care,0 days 00:00:01.811046,0 days 00:01:49.907483,0 days 00:01:48.096437
3,9,weekly_report.care,0 days 00:00:00.020072,0 days 00:01:48.116043,0 days 00:01:48.095971
4,8,weekly_report.care,0 days 00:00:00.013578,0 days 00:01:48.109265,0 days 00:01:48.095687


In [13]:
print('Событие с максимальным разбросом времени его наступления:')
print('event_id:', events_disp.loc[0, 'event_id'])
print('sender:', events_disp.loc[0, 'sender'])
print('Название события:', event_name_12.loc[event_name_12['id'] == events_disp.loc[0, 'event_id'],'event_name'])

Событие с максимальным разбросом времени его наступления:
event_id: 20
sender: weekly_report.care
Название события: 19    Выполняем прогноз временного ряда
Name: event_name, dtype: object


Исследование показало, что максимальный разброс времени наблюдается до наступления события прогнозирования временного ряда: событие 20, sender - weekly_report.care. Скорее всего, это связано с продолжительностью получения и предобработки данных, а также с настройкой модели (например, получение гиперпарамеров и регрессоров).

## Вопрос 3

**Какая задача прогнозирования выполняется дольше всего (название задачи)?**

In [14]:
# создадим сводную таблицу, содержащую времея начала и окончания каждой задачи
sessions_time = events.pivot_table(index=['session_id', 'forecastMarker'],
                                  values='request_time', 
                                  aggfunc=['min','max'])
# переименуем столбцы сводной таблицы
sessions_time.columns = ['start', 'stop']
# добавим столбец с продолжительностью каждой задачи
sessions_time['duration'] = sessions_time['stop'] - sessions_time['start']
# отсортируем таблицу по убыванию продолжительности выполнения задачи
sessions_time = sessions_time.sort_values(by='duration', ascending=False).reset_index()
sessions_time.head(5)

,session_id,forecastMarker,start,stop,duration
0,4dcab2af4573ced8251fb7433174ba15,weekly_report.care:count_of_jivo_SA,2022-09-14 05:20:52.203702,2022-09-14 05:22:44.688175,0 days 00:01:52.484473
1,f151aa3db81db96c276313c910abca75,weekly_report.SAAS:visitors,2022-09-12 17:08:06.244552,2022-09-12 17:09:19.750563,0 days 00:01:13.506011
2,7714ae1e594b3b4cb64cae7a4607bcce,weekly_report.care:sum_of_income,2022-09-14 05:18:49.192361,2022-09-14 05:20:01.750742,0 days 00:01:12.558381
3,f96f949df754e973ae7ed08f79c64c6c,weekly_report.SAAS:visitors,2022-09-12 17:03:42.199231,2022-09-12 17:04:53.517896,0 days 00:01:11.318665
4,e35e3228f80531fbc06572a0baf07c11,weekly_report.care:count_of_jivo_SS,2022-09-13 08:59:32.813027,2022-09-13 09:00:42.937581,0 days 00:01:10.124554


In [15]:
print('Самая продолжительная задача:')
print('session_id:', sessions_time.loc[0, 'session_id'])
print('forecastMarker:', sessions_time.loc[0, 'forecastMarker'])
print('Время выполнения:',  sessions_time.loc[0, 'duration'])

Самая продолжительная задача:
session_id: 4dcab2af4573ced8251fb7433174ba15
forecastMarker: weekly_report.care:count_of_jivo_SA
Время выполнения: 0 days 00:01:52.484473


Можно заметить, что самой продолжительной задачей является задача 4dcab2af4573ced8251fb7433174ba15, которая также обладает наибольшим шагом между событиями 6 и 7 (мы обнаружили это при исследовании вопроса №1). Для этой задачи время выполнения события 6 (чтение данных из BQ по полученному SQL запросу) занимает 1 минуту 46.5 секунд, а все остальные события выполняются менее, чем за 6 секунд. При этом задача заканчивается успешно - присутсвует идентификатор события 31. Это значит, что при оптимизации операции чтения данных по SQL запросу, эта задача, и, возможно, другие, будут выполняться ещё быстрее.

## Вопрос 4

**Какое количество задач прогнозирования могут выполняться одновременно? Укажите максимальное число параллельных задач.**

Воспользуемся сводной таблицей sessions_time из предыдущего вопроса. 

In [16]:
# определим переменные для цикла
max_task_num = 1

for index in range (0,len(sessions_time)):
    # определим время начала и окончания выполнения задачи
    start_time = sessions_time.loc[index, 'start']
    stop_time = sessions_time.loc[index, 'stop']
    # выделим все задачи, которые начали исполняться в промежутке времени между start_time и stop_time
    task_num = len(sessions_time.query('@start_time <= start <= @stop_time'))
    # сохраним максимальное количество параллельных задач
    if task_num > max_task_num:
        max_task_num = task_num

In [17]:
print('Максимальное число параллельных задач:', max_task_num)

Максимальное число параллельных задач: 10


## Вопрос 5

**Перечислите forecastMarker которые не завершились успешно**

Задача прогнозирования считается завершенной успешно, если наступило событие с идентификатором 31.

In [18]:
# создадим сводную таблицу, содержащую последний event_id для каждого идентификатора задачи и forecastMarker
pivot_forecastMarker = events.pivot_table(index=['forecastMarker', 'session_id'], values='event_id', aggfunc='last')
# переименуем столбцы
pivot_forecastMarker.columns = ['last_event_id']

# выведем строки, в которых последнее значение event_id не равно 31
pivot_forecastMarker.query('last_event_id != 31')

,,last_event_id
forecastMarker,session_id,
None,56af71228bfbb91afd00a04652646069,32
SaaS/RUS/turnover/targetMailRu,05fde45de80ea82b620d924acf35410c,32
weekly_report.SAAS:new_visitors,ef2ce869aedbddee01530295d44a5993,32
weekly_report.SAAS:new_visitors:utm_medium:organic,56e957cb0972c856c211dfed092dc8c2,25
weekly_report.SAAS:turnover,9dd140e965fe9c98656a0d0e405aa1e8,20
